#### Imports

* Linear operator implementation.
* Iterative system solver.

In [1]:
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import gmres

# Vortex solver

An iterative linear operator will be used. This will not affect our solution, but will simplify the way we build the matrix. 

In [5]:
def build_RHS(alpha):
    """
    Build the linear system RHS.
    
    Parameters
    ----------
    alpha: float
        Degrees.
        
    Returns
    -------
    np.array
    """
    _alpha = np.deg2rad(alpha)

    # RHS
    b = np.zeros(N_b)

    for idx in range(len(b)-1):
        b[idx] = np.sin(theta[idx] - _alpha)

    # Kutta condition
    b[-1] = 0.0
    
    return b

In [6]:
def compute_velocity(gamma, alpha):
    """
    Compute the velocity magnitude at each control point.
    
    Parameters
    ----------
    gamma: np.array
    alpha: float
        Degrees.
        
    Returns
    -------
    np.array
    """
    _alpha = np.deg2rad(alpha)
    
    # Prepare vector
    result = np.zeros(len(gamma)-1)
    
    # Matrix-product implementation
    for i in range(len(result)):
        for j in range(len(gamma)-1):
            
            result[i] += contribution_vortex_tangential(i, j, gamma)
    
    return np.cos(theta - _alpha) + result

In [7]:
def compute_pressure_distribution(gamma, alpha):
    """
    Compute the pressure distribution.
    
    Parameters
    ----------
    gamma: np.array
    alpha: float
        Degrees.
    
    Returns
    -------
    np.array
    """
    velocity = compute_velocity(gamma, alpha)
    
    return 1.0 - np.power(velocity, 2.0)

In [8]:
def compute_lift(gamma, alpha):
    """
    Compute the lift coefficient.
    
    Parameters
    ----------
    gamma: np.array
    alpha: float
        Degrees.
    
    Returns
    -------
    np.array
    """
    cp    = compute_pressure_distribution(gamma, alpha)
    vec_j = np.array([0, 1])
     
    return np.sum(-cp * np.dot(normals, vec_j) * S)

In [9]:
def solve(alpha):
    """
    Solve the airfoil vortex sheets to obtain the
    pressure distribution and the lift coefficient.
    
    Parameters
    ----------
    alpha: float
        Degrees
        
    Returns
    -------
    gammas: np.array
    cp:     np.array
    cl:     float
    
    Notes
    -----
    The vortex strength is given in a dimensionless way:
    
    ```math
    \gamma = \frac{\gamma}{2 \pi V_{\infty}}
    ```
    """
    A = LinearOperator((N_b, N_b), matvec=gammas_matrix_equations)
    b = build_RHS(alpha)

    gammas, info = gmres(A, b, 
                         x0      = np.ones(N_b), 
                         restart = ITERATIONS_RESTART, 
                         maxiter = ITERATIONS_MAX, 
                         tol     = 1e-8,
                         atol    = 1e-8)
    
    # Check for convergence
    if info != 0:
        
        raise ValueError('The system did not converge.')
        
    else:
        
        cp = compute_pressure_distribution(gamma=gammas, alpha=alpha)
        cl = compute_lift                 (gamma=gammas, alpha=alpha)
        
        return gammas, cp, cl